# Setup

Import libraries

In [1]:
import numpy as np
import tensorflow as tf
from os import chdir, getcwd, listdir, mkdir
from os.path import join
import requests
from xml.etree import ElementTree as ET
import re
import pandas as pd
import time
import pkg_resources

HOME = getcwd()

In [2]:
package_list = [pkg for pkg in pkg_resources.working_set]

if 'googletrans' not in [pkg.key for pkg in package_list]:
    ! pip install googletrans==3.1.0a0
import googletrans
    
if 'pdfplumber' not in [pkg.key for pkg in package_list]:
    ! pip install pdfplumber
import pdfplumber

## FAR Part 121

Download file from the ECFR website

In [ ]:
URL = 'https://www.govinfo.gov/bulkdata/ECFR/title-14/ECFR-title14.xml'

if 'Regulations' not in listdir():
    mkdir('Regulations')

chdir(join(HOME, 'Regulations'))
r = requests.get(URL)
with open('ECFR-title14.xml', 'wb') as file:
    file.write(r.content)

chdir(HOME)

Identify node corresponding to Part 121 xml file

In [ ]:
filepath = join('Regulations', 'ECFR-title14.xml')
tree = ET.parse(filepath)
root = tree.getroot()

for element in root.iter():
    if element.tag == 'DIV5':
        if element.attrib['N'] == '121':
            root121 = element

section_nodes = []

for element in root121.iter():
    if element.tag == 'DIV8':
        section_nodes.append(element)

Save file with individual sections:

In [ ]:
titles = []
sections = []

for section in section_nodes:
    titles.append(section.attrib['N'])
    text = ''
    for line in section.itertext():
        if re.search('\w', line):
            text += line
    sections.append(text)
    
filename = join(HOME, 'Regulations', 'FAR_Part121_sections.xlsx')    
pd.DataFrame({'title':titles, 'requirement':sections}).to_excel(filename)

Test splitting method: print section to screen:

In [ ]:
SECTION_NUMBER = 155

section, paragraph, item = '', '', ''
section_id, paragraph_id, item_id = '', '', ''
current_level = ''

for line in section_nodes[SECTION_NUMBER].itertext():
    if re.match('^§ 121.\d*', line):
        current_level = 'section'
        section_id = re.findall('^§ 121.\d*', line)[0]
        section = line + '\n'
        paragraph, item, paragraph_id, item_id = '', '', '', ''
    elif re.match('^\([a-h]\)', line):
        if current_level == 'paragraph' or current_level == 'item':
            print(section_id + paragraph_id + item_id + '\n')
            print(section + paragraph + item)
            print('----------')
        paragraph_id = re.findall('^\([a-h]\)', line)[0]
        paragraph = line
        item, item_id = '', ''
        current_level = 'paragraph'
    elif re.match('^\(\d*\)',line):
        if current_level == 'item':
            print(section_id + paragraph_id + item_id + '\n')
            print(section + paragraph + item)
            print('----------')
        item_id = re.findall('^\(\d*\)',line)[0]
        item = line
        current_level = 'item'
    elif re.match('^\s+$', line):
        pass
    elif re.match('\[.*\]', line):
        pass
    else:
        if current_level == 'section':
            section += line
        elif current_level == 'paragraph':
            paragraph += line
        elif current_level == 'item':
            item += line
print(section_id + paragraph_id + item_id + '\n')
print(section + paragraph + item)

Create lists of titles and requirements:

In [ ]:
requirement_titles = []
requirement_texts = []

for sec in section_nodes:
    
    section, paragraph, item = '', '', ''
    section_id, paragraph_id, item_id = '', '', ''
    current_level = ''

    for line in sec.itertext():
        if re.match('^§ 121.\d*', line):
            current_level = 'section'
            section_id = re.findall('^§ 121.\d*', line)[0]
            section = line + '\n'
            paragraph, item, paragraph_id, item_id = '', '', '', ''
        elif re.match('^\([a-h]\)', line):
            if current_level == 'paragraph' or current_level == 'item':
                requirement_titles.append(section_id + paragraph_id + item_id)
                requirement_texts.append(section + paragraph + item)
            paragraph_id = re.findall('^\([a-h]\)', line)[0]
            paragraph = line
            item, item_id = '', ''
            current_level = 'paragraph'
        elif re.match('^\(\d*\)',line):
            if current_level == 'item':
                requirement_titles.append(section_id + paragraph_id + item_id)
                requirement_texts.append(section + paragraph + item)
            item_id = re.findall('^\(\d*\)',line)[0]
            item = line
            current_level = 'item'
        elif re.match('^\s+$', line):
            pass
        elif re.match('\[.*\]', line):
            pass
        else:
            if current_level == 'section':
                section += line
            elif current_level == 'paragraph':
                paragraph += line
            elif current_level == 'item':
                item += line
    requirement_titles.append(section_id + paragraph_id + item_id)
    requirement_texts.append(section + paragraph + item)

Save excel file to disk:

In [ ]:
filepath = join(HOME, 'Regulations', 'FAR_Part121_nodes.xlsx')
df = pd.DataFrame(zip(requirement_titles, requirement_texts), columns = ['title', 'requirement'])
df = df.loc[df.title != '']
df.to_excel(filepath)

## RBAC 121

Download the pdf file from www.ANAC.gov.br

In [3]:
url = 'https://www.anac.gov.br/assuntos/legislacao/legislacao-1/rbha-e-rbac/rbac/rbac-121/@@display-file/arquivo_norma/RBAC121EMD12.pdf'

r = requests.get(url)

if 'Regulations' not in listdir():
    mkdir('Regulations')

chdir(join(HOME, 'Regulations'))

with open('RBAC121.pdf', 'wb') as file:
    file.write(r.content)

chdir(HOME)

Parse the .pdf file and save to .txt

In [4]:
filepath = join(HOME, 'Regulations', 'RBAC121.pdf')
text = ''
chdir(HOME)

with pdfplumber.open(filepath) as pdf:
    #for page in range(len(pdf.pages)):
    for page in range(9,243): # skip the table of contents and stop before annexes
        text += pdf.pages[page].extract_text()
        
filepath = join(HOME, 'Regulations', 'RBAC121.txt')
with open(filepath, 'w') as file:
    file.write(text)

### Save version with individual sections

Create list of individual sections

In [5]:
filepath = join(HOME, 'Regulations', 'RBAC121.txt')

requirements = []
titles = []

with open(filepath) as file:
    for line in file:
        if (re.match(' Data da emissão', line) or 
            re.match('Data de vigência', line) or 
            re.match('Origem: SPO', line) or
            #re.match('SUBPARTE ', line) or
            #re.match('\[RESERVADO\]', line) or
            not re.search('\w', line)):
            pass
        elif re.match('^121\.\d{1,4}  ', line):
            requirements.append(line)
        elif len(requirements) > 0:
            requirements[-1] += line
            
for n, req in enumerate(requirements):
    #titles.append(re.findall('121.\d{1,4}', req)[0]) # alternative implementation
    titles.append(req.split('  ')[0])
    requirements[n] = re.sub('\nSUBPARTE \w*\s\n.*$', '', req, flags = re.DOTALL)
    
filepath = join(HOME, 'Regulations', 'RBAC121_sections.xlsx')    
pd.DataFrame({'title':titles, 'requirement':requirements}).to_excel(filepath)

### Save split version

Split sections into paragraphs and items and save to excel

In [116]:
filepath = join(HOME, 'Regulations', 'RBAC121_sections.xlsx')
df_sections = pd.read_excel(filepath, dtype=str)[['title', 'requirement']]

section_titles, section_texts = [], []
item_titles, item_texts = [], []


for section_tuple in df_sections.itertuples():
    sec = section_tuple[1]
    req = section_tuple[2]
    section, paragraph, item = '', '', ''
    section_id, paragraph_id, item_id = '', '', ''
    current_level = ''
    
    for line in req.split('\n'):
        if re.match('^121.\d*', line) and re.match(req.split(' ')[0], line):
            current_level = 'section'
            section_id = re.findall('^121.\d*', line)[0]
            section = line + '\n'
            paragraph, item, paragraph_id, item_id = '', '', '', ''
        elif re.match('^\([a-h]\)', line):
            if current_level == 'paragraph' or current_level == 'item':
                section_titles.append(section_id + paragraph_id + item_id)
                section_texts.append(section + paragraph + item)
            paragraph_id = re.findall('^\([a-h]\)', line)[0]
            paragraph = line
            item, item_id = '', ''
            current_level = 'paragraph'
        elif re.match('^\(\d*\)',line):
            if current_level == 'item':
                section_titles.append(section_id + paragraph_id + item_id)
                section_texts.append(section + paragraph + item)
            item_id = re.findall('^\(\d*\)',line)[0]
            item = line
            current_level = 'item'
        elif re.match('^\s+$', line):
            pass
        elif re.match('\[.*\]', line):
            pass
        else:
            if current_level == 'section':
                section += line
            elif current_level == 'paragraph':
                paragraph += line
            elif current_level == 'item':
                item += line
    section_titles.append(section_id + paragraph_id + item_id)
    section_texts.append(section + paragraph + item)

In [119]:
filepath = join(HOME, 'Regulations', 'RBAC121_nodes.xlsx')
df_nodes = pd.DataFrame({'title':section_titles, 'requirement':section_texts})
df_nodes.to_excel(filepath)

### Translation

Create lists containing each section translation and translation status

In [123]:
filepath = join(HOME, 'Regulations', 'RBAC121_sections.xlsx')
requirements = pd.read_excel(filepath)['requirement'].tolist()

True

In [12]:
RBAC121_lang = np.repeat(np.array('pt'), len(requirements)).tolist()
RBAC121_en = np.repeat(np.array(''), len(requirements)).tolist()
RBAC121_extra_data = np.repeat(np.array(''), len(requirements)).tolist()

Translate each section using the google translate service. Run this code every few hours until all sentences are translated:

In [13]:
translator = googletrans.Translator()
wait_delay = 2 * 60 * 60 # 1 hours

In [ ]:
wait_bool = False
translation_ready = 'RBAC121_sections_en.xlsx' in listdir(join(HOME, 'Regulations'))

while translation_ready != True:
    
    if wait_bool: # do not wait on the first time
        time.sleep(wait_delay)
    wait_bool=True
    
    for section in range(len(requirements)):
        if RBAC121_lang[section] == 'pt':
            translation = translator.translate(requirements[section], dest='en', src='pt')
            RBAC121_en[section] = translation.text
            RBAC121_extra_data[section] = translation.extra_data
            if translation.extra_data['original-language'] == 'pt':
                RBAC121_lang[section] = 'en'

    print('Translated sentences: {} out of {}'.format(RBAC121_lang.count('en'), len(RBAC121_lang)))
    if RBAC121_lang.count('en') == len(RBAC121_lang):
        translation_ready = True

Translated sentences: 0 out of 360
Translated sentences: 132 out of 360
Translated sentences: 259 out of 360
Translated sentences: 358 out of 360
Translated sentences: 358 out of 360


Save the full translation to .txt file.

In [ ]:
filepath = join('Regulations', 'RBAC121_sections_en.xlsx')
df = pd.DataFrame({'title':titles, 'requirement':requirements, 'translation':RBAC121_en})
df.to_excel(filepath)

### Prepare validation data from ANAC
* Load ANAC requirements
* Copy label from FAA requirement with same number
* Filter out requirements without label
* Take sample of 100 requirements
* Translate remaining requirements

In [3]:
df_ANAC = pd.read_excel(join(HOME, 'Regulations', 'RBAC121_nodes.xlsx'))[['title', 'requirement']]
df_FAA = pd.read_excel(join(HOME, 'Regulations', 'FAR_Part121_nodes_labelled.xlsx'))[['title', 'requirement', 'label', 'tag']]

df_FAA.title = df_FAA.title.map(lambda t: t[2:])
df_ANAC = df_ANAC.join(df_FAA[['title', 'label', 'tag']].set_index('title'), on = 'title', how = 'left')
df_ANAC.requirement = df_ANAC.requirement.map(lambda t: t.replace('\n', ' '))

In [4]:
translations = []
reqs = df_ANAC.requirement.tolist()
translator = googletrans.Translator()
i = 0

while i < len(reqs):
    translation = translator.translate(reqs[i], src = 'pt', dest = 'en')
    
    if translation.src == 'pt':
        print('translation {} of {} OK!'.format(i, len(reqs)))
        translations.append(translation.text)
        i += 1
    else:
        print('translation {} of {} not OK... going to sleep at {}'.format(i, len(reqs), time.strftime('%H:%M (%d/%m)')))
        time.sleep(3 * 60 * 60)

translation 0 of 2132 OK!
translation 1 of 2132 OK!
translation 2 of 2132 OK!
translation 3 of 2132 OK!
translation 4 of 2132 OK!
translation 5 of 2132 OK!
translation 6 of 2132 OK!
translation 7 of 2132 OK!
translation 8 of 2132 OK!
translation 9 of 2132 OK!
translation 10 of 2132 OK!
translation 11 of 2132 OK!
translation 12 of 2132 OK!
translation 13 of 2132 OK!
translation 14 of 2132 OK!
translation 15 of 2132 OK!
translation 16 of 2132 OK!
translation 17 of 2132 OK!
translation 18 of 2132 OK!
translation 19 of 2132 OK!
translation 20 of 2132 OK!
translation 21 of 2132 OK!
translation 22 of 2132 OK!
translation 23 of 2132 OK!
translation 24 of 2132 OK!
translation 25 of 2132 OK!
translation 26 of 2132 OK!
translation 27 of 2132 OK!
translation 28 of 2132 OK!
translation 29 of 2132 OK!
translation 30 of 2132 OK!
translation 31 of 2132 OK!
translation 32 of 2132 OK!
translation 33 of 2132 OK!
translation 34 of 2132 OK!
translation 35 of 2132 OK!
translation 36 of 2132 OK!
translation

translation 295 of 2132 OK!
translation 296 of 2132 OK!
translation 297 of 2132 OK!
translation 298 of 2132 OK!
translation 299 of 2132 OK!
translation 300 of 2132 OK!
translation 301 of 2132 OK!
translation 302 of 2132 OK!
translation 303 of 2132 OK!
translation 304 of 2132 OK!
translation 305 of 2132 OK!
translation 306 of 2132 OK!
translation 307 of 2132 OK!
translation 308 of 2132 not OK... going to sleep at 22:29 (23/04)
translation 308 of 2132 OK!
translation 309 of 2132 OK!
translation 310 of 2132 OK!
translation 311 of 2132 OK!
translation 312 of 2132 OK!
translation 313 of 2132 OK!
translation 314 of 2132 OK!
translation 315 of 2132 OK!
translation 316 of 2132 OK!
translation 317 of 2132 OK!
translation 318 of 2132 OK!
translation 319 of 2132 OK!
translation 320 of 2132 OK!
translation 321 of 2132 OK!
translation 322 of 2132 OK!
translation 323 of 2132 OK!
translation 324 of 2132 OK!
translation 325 of 2132 OK!
translation 326 of 2132 OK!
translation 327 of 2132 OK!
translatio

translation 581 of 2132 OK!
translation 582 of 2132 OK!
translation 583 of 2132 OK!
translation 584 of 2132 OK!
translation 585 of 2132 OK!
translation 586 of 2132 OK!
translation 587 of 2132 OK!
translation 588 of 2132 OK!
translation 589 of 2132 OK!
translation 590 of 2132 OK!
translation 591 of 2132 OK!
translation 592 of 2132 OK!
translation 593 of 2132 OK!
translation 594 of 2132 OK!
translation 595 of 2132 OK!
translation 596 of 2132 OK!
translation 597 of 2132 OK!
translation 598 of 2132 OK!
translation 599 of 2132 OK!
translation 600 of 2132 OK!
translation 601 of 2132 OK!
translation 602 of 2132 OK!
translation 603 of 2132 OK!
translation 604 of 2132 OK!
translation 605 of 2132 OK!
translation 606 of 2132 OK!
translation 607 of 2132 OK!
translation 608 of 2132 OK!
translation 609 of 2132 OK!
translation 610 of 2132 OK!
translation 611 of 2132 OK!
translation 612 of 2132 OK!
translation 613 of 2132 OK!
translation 614 of 2132 OK!
translation 615 of 2132 OK!
translation 616 of 2

translation 869 of 2132 OK!
translation 870 of 2132 OK!
translation 871 of 2132 OK!
translation 872 of 2132 OK!
translation 873 of 2132 OK!
translation 874 of 2132 OK!
translation 875 of 2132 OK!
translation 876 of 2132 OK!
translation 877 of 2132 OK!
translation 878 of 2132 OK!
translation 879 of 2132 OK!
translation 880 of 2132 OK!
translation 881 of 2132 OK!
translation 882 of 2132 OK!
translation 883 of 2132 OK!
translation 884 of 2132 OK!
translation 885 of 2132 OK!
translation 886 of 2132 OK!
translation 887 of 2132 OK!
translation 888 of 2132 OK!
translation 889 of 2132 OK!
translation 890 of 2132 OK!
translation 891 of 2132 OK!
translation 892 of 2132 OK!
translation 893 of 2132 OK!
translation 894 of 2132 OK!
translation 895 of 2132 OK!
translation 896 of 2132 OK!
translation 897 of 2132 OK!
translation 898 of 2132 OK!
translation 899 of 2132 OK!
translation 900 of 2132 OK!
translation 901 of 2132 OK!
translation 902 of 2132 OK!
translation 903 of 2132 OK!
translation 904 of 2

translation 1152 of 2132 OK!
translation 1153 of 2132 OK!
translation 1154 of 2132 OK!
translation 1155 of 2132 OK!
translation 1156 of 2132 OK!
translation 1157 of 2132 OK!
translation 1158 of 2132 OK!
translation 1159 of 2132 OK!
translation 1160 of 2132 OK!
translation 1161 of 2132 OK!
translation 1162 of 2132 OK!
translation 1163 of 2132 OK!
translation 1164 of 2132 OK!
translation 1165 of 2132 not OK... going to sleep at 19:34 (24/04)
translation 1165 of 2132 OK!
translation 1166 of 2132 OK!
translation 1167 of 2132 OK!
translation 1168 of 2132 OK!
translation 1169 of 2132 OK!
translation 1170 of 2132 OK!
translation 1171 of 2132 OK!
translation 1172 of 2132 OK!
translation 1173 of 2132 OK!
translation 1174 of 2132 OK!
translation 1175 of 2132 OK!
translation 1176 of 2132 OK!
translation 1177 of 2132 OK!
translation 1178 of 2132 OK!
translation 1179 of 2132 OK!
translation 1180 of 2132 OK!
translation 1181 of 2132 OK!
translation 1182 of 2132 OK!
translation 1183 of 2132 OK!
trans

translation 1428 of 2132 OK!
translation 1429 of 2132 OK!
translation 1430 of 2132 OK!
translation 1431 of 2132 OK!
translation 1432 of 2132 OK!
translation 1433 of 2132 OK!
translation 1434 of 2132 OK!
translation 1435 of 2132 OK!
translation 1436 of 2132 OK!
translation 1437 of 2132 OK!
translation 1438 of 2132 OK!
translation 1439 of 2132 OK!
translation 1440 of 2132 OK!
translation 1441 of 2132 OK!
translation 1442 of 2132 OK!
translation 1443 of 2132 OK!
translation 1444 of 2132 OK!
translation 1445 of 2132 OK!
translation 1446 of 2132 OK!
translation 1447 of 2132 OK!
translation 1448 of 2132 OK!
translation 1449 of 2132 OK!
translation 1450 of 2132 OK!
translation 1451 of 2132 OK!
translation 1452 of 2132 OK!
translation 1453 of 2132 OK!
translation 1454 of 2132 OK!
translation 1455 of 2132 OK!
translation 1456 of 2132 OK!
translation 1457 of 2132 OK!
translation 1458 of 2132 OK!
translation 1459 of 2132 OK!
translation 1460 of 2132 OK!
translation 1461 of 2132 OK!
translation 14

translation 1706 of 2132 OK!
translation 1707 of 2132 OK!
translation 1708 of 2132 OK!
translation 1709 of 2132 OK!
translation 1710 of 2132 OK!
translation 1711 of 2132 OK!
translation 1712 of 2132 OK!
translation 1713 of 2132 OK!
translation 1714 of 2132 OK!
translation 1715 of 2132 OK!
translation 1716 of 2132 OK!
translation 1717 of 2132 OK!
translation 1718 of 2132 OK!
translation 1719 of 2132 OK!
translation 1720 of 2132 OK!
translation 1721 of 2132 OK!
translation 1722 of 2132 OK!
translation 1723 of 2132 OK!
translation 1724 of 2132 OK!
translation 1725 of 2132 OK!
translation 1726 of 2132 OK!
translation 1727 of 2132 OK!
translation 1728 of 2132 OK!
translation 1729 of 2132 OK!
translation 1730 of 2132 OK!
translation 1731 of 2132 OK!
translation 1732 of 2132 OK!
translation 1733 of 2132 OK!
translation 1734 of 2132 OK!
translation 1735 of 2132 OK!
translation 1736 of 2132 OK!
translation 1737 of 2132 OK!
translation 1738 of 2132 OK!
translation 1739 of 2132 OK!
translation 17

translation 1984 of 2132 OK!
translation 1985 of 2132 OK!
translation 1986 of 2132 OK!
translation 1987 of 2132 OK!
translation 1988 of 2132 OK!
translation 1989 of 2132 OK!
translation 1990 of 2132 OK!
translation 1991 of 2132 OK!
translation 1992 of 2132 OK!
translation 1993 of 2132 OK!
translation 1994 of 2132 OK!
translation 1995 of 2132 OK!
translation 1996 of 2132 OK!
translation 1997 of 2132 OK!
translation 1998 of 2132 OK!
translation 1999 of 2132 OK!
translation 2000 of 2132 OK!
translation 2001 of 2132 OK!
translation 2002 of 2132 OK!
translation 2003 of 2132 OK!
translation 2004 of 2132 OK!
translation 2005 of 2132 OK!
translation 2006 of 2132 OK!
translation 2007 of 2132 OK!
translation 2008 of 2132 OK!
translation 2009 of 2132 OK!
translation 2010 of 2132 OK!
translation 2011 of 2132 not OK... going to sleep at 16:39 (25/04)
translation 2011 of 2132 OK!
translation 2012 of 2132 OK!
translation 2013 of 2132 OK!
translation 2014 of 2132 OK!
translation 2015 of 2132 OK!
trans

In [ ]:
filepath = join(HOME, 'Regulations', 'RBAC121_nodes_en_partially_labelled.xlsx')
df_ANAC.insert(loc = 2, column = 'requirement_en', value = translations)
df_ANAC.to_excel(filepath)

In [13]:
df_ANAC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2132 entries, 0 to 2071
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2132 non-null   object 
 1   requirement     2132 non-null   object 
 2   requirement_en  2132 non-null   object 
 3   label           1779 non-null   float64
 4   tag             1779 non-null   object 
dtypes: float64(1), object(4)
memory usage: 99.9+ KB


# Trash

In [110]:
#translations = []
#language = []
#translator = googletrans.Translator()

#for req in df_ANAC.requirement:
#    translation = translator.translate(req, dest='en', src='pt')
#    language.append(translation.src)
#    translations.append(translation.text)

KeyboardInterrupt: 